In [10]:
from src.utils.load_data import load_data
from src.utils.stratified_train_test_split import stratified_train_test_split
from src.utils.IntervalsHandler import IntervalsHandler
from src.utils.plot import plot
from src.utils.metric_list import metric_list

from src.models.BasicClassificationModel import BasicClassificationModel
from src.models.AdvancedClassificationModel import AdvancedClassificationModel
from src.models.BasicRegressionModel import BasicRegressionModel
from src.models.AdvancedRegressionModel import AdvancedRegressionModel

from src.evaluation.AbstractEvaluationResults import MetricEnum

In [11]:
interval_functions = [
    lambda x: 10 ** (0.66 * x) + 1500 * (x + 1) ** 1.3 + 800 * x ** 1.1,
    lambda x: 10 ** (0.25 * x) + 600 * (x + 1) ** 1.15 + 200 * x ** 0.75,
    lambda x: 10 ** (0.19 * x) + 80 * (x + 1) ** 1.02 + 150 * x ** 0.4,
    lambda x: 10 ** (0.13 * x) + 50 * (x + 1) ** 1.01 + 100 * x ** 0.35,
    lambda x: 10 ** (0.1 * x) + 30 * (x + 1) ** 1.005 + 90 * x ** 0.3,
    lambda x: 10 ** (0.08 * x) + 15 * (x + 1) ** 1.001 + 60 * x ** 0.3,
    lambda x: 10 ** (0.06 * x) + 15 * (x + 1) ** 1.001 + 40 * x ** 0.25,
    lambda x: 10 ** (0.05 * x) + 10 * (x + 1) ** 1.002 + 30 * x ** 0.2,
]

In [12]:
X, y = load_data()

# Regression
X_train, X_test, y_train, y_test = stratified_train_test_split(X, y, 0.3, 42)

# Classification
intervals_handlers = [IntervalsHandler(y, func) for func in interval_functions]

for handler in intervals_handlers:
    handler.classify()
    handler.reduction()

intervals_numbers = [handler.intervals_number for handler in intervals_handlers] 

y_classified = [handler.data for handler in intervals_handlers]

classified_data = [stratified_train_test_split(X, y_class, 0.3, 42) for y_class in y_classified]
X_train_class, X_test_class, y_train_class, y_test_class = zip(*classified_data)

X_train_class = list(X_train_class)
X_test_class = list(X_test_class)
y_train_class = list(y_train_class)
y_test_class = list(y_test_class)

In [13]:

basic_regression_model = BasicRegressionModel()
basic_regression_model.load_hyperparameters("basic_regression")
basic_regression_model.fit(X_train, y_train)
basic_regression_score = [
    basic_regression_model.score_classification(*test_data)
    for test_data in zip(X_test_class, y_test_class, intervals_handlers)
]

In [14]:
advanced_regression_model = AdvancedRegressionModel()
advanced_regression_model.load_hyperparameters("advanced_regression")
advanced_regression_model.fit(X_train, y_train)
advanced_regression_score = [
    advanced_regression_model.score_classification(*test_data)
    for test_data in zip(X_test_class, y_test_class, intervals_handlers)
]

In [15]:
basic_classification_score = []
for _X_train, _X_test, _y_train, _y_test in classified_data:
    basic_classification_model = BasicClassificationModel()
    basic_classification_model.load_hyperparameters("basic_classification")
    basic_classification_model.fit(_X_train, _y_train)
    basic_classification_score.append(basic_classification_model.score(_X_test, _y_test))

In [16]:
advanced_classification_score = []
for _X_train, _X_test, _y_train, _y_test in classified_data:
    advanced_classification_model = AdvancedClassificationModel()
    advanced_classification_model.load_hyperparameters("advanced_classification")
    advanced_classification_model.fit(_X_train, _y_train)
    advanced_classification_score.append(advanced_classification_model.score(_X_test, _y_test))

In [ ]:
plot(
    intervals_numbers,
    {
        "Basic Classification": metric_list(basic_classification_score, MetricEnum.ACCURACY),
        "Advanced Classification": metric_list(advanced_classification_score, MetricEnum.ACCURACY),
        "Basic Regression": metric_list(basic_regression_score, MetricEnum.ACCURACY),
        "Advanced Regression": metric_list(advanced_regression_score, MetricEnum.ACCURACY),
    },
    "Models Comprasion",
    "Number of Intervals",
    "Accuracy"
)

In [ ]:
plot(
    intervals_numbers,
    {
        "Basic Classification": metric_list(basic_classification_score, MetricEnum.MEAN_CLASTERS_ERROR),
        "Advanced Classification": metric_list(advanced_classification_score, MetricEnum.MEAN_CLASTERS_ERROR),
        "Basic Regression": metric_list(basic_regression_score, MetricEnum.MEAN_CLASTERS_ERROR),
        "Advanced Regression": metric_list(advanced_regression_score, MetricEnum.MEAN_CLASTERS_ERROR),
    },
    "Models Comprasion",
    "Number of Intervals",
    "Mean Clusters Error"
)